In [1]:
!pip install datasets -q

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None


In [2]:
!pip install sagemaker -U -q

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None


In [3]:
!pip install s3fs==0.4.2 -U -q

distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None


### Load dataset and have a peak:

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)
df = pd.read_csv('data/ade.csv')
df.sample(5, random_state=124)

,text,label
215,"Although the literature on the use of risperidone in elderly patients with dementia consists largely of uncontrolled trials, case reports, and chart reviews, it appears that this agent is effective for managing agitation in this population and does so with a low frequency of extrapyramidal symptoms (EPS).",1
10951,Cyproterone acetate (CPA) is a steroidal synthetic progestagen and anti-androgenic compound widely administered in prostate cancer which has been evidentially correlated with a severe hepatotoxic potency.,0
155,"The presented patient was treated with 200 mg TCA and developed Cushing's syndrome 6 weeks later (cortisol and ACTH concentrations were below limits of detection, TCA concentrations were > 3 micrograms/l).",1
8412,After completion of chemotherapy the patient remained in a complete remission and free of new thromboembolic events.,0
10741,"A liver biopsy shortly after abstinence showed extensive pericellular fibrosis, but a biopsy after 6 years showed very mild fibrosis.",0


### Determine ratio of positive ADE phrases compared to total dataset

In [6]:
df['label'].sum()/len(df)

0.20439318529862174

### Initialise Sagemaker variables and copy data into S3 bucket

In [7]:
from sagemaker.huggingface.processing import HuggingFaceProcessor
import sagemaker
from sagemaker import get_execution_role

In [8]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = f"az-ade-{sess.account_id()}"
sess._create_s3_bucket_if_it_does_not_exist(bucket_name=bucket, region=sess._region_name)
filepath = f"s3://{bucket}/data/ade.csv"

In [9]:
filepath

's3://az-ade-905847418383/data/ade.csv'

In [10]:
!aws s3 cp data/ade.csv $filepath

upload: data/ade.csv to s3://az-ade-905847418383/data/ade.csv    


### Save the name of the S3 bucket for later sessions

In [11]:
%store bucket

Stored 'bucket' (str)


### Set up processing job

In [12]:
hf_processor = HuggingFaceProcessor(
    role=role,
    instance_type="ml.p3.2xlarge",
    transformers_version='4.6',
    base_job_name="az-ade",
    pytorch_version='1.7',
    instance_count=1,
)

In [13]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/training", destination=f"s3://{bucket}/processing_output/train_data"),
        ProcessingOutput(output_name="validation_data", source="/opt/ml/processing/validation", destination=f"s3://{bucket}/processing_output/validation_data"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test", destination=f"s3://{bucket}/processing_output/test_data"),
    ]
arguments = ["--dataset-name", "ade_corpus_v2",
             "--datasubset-name", "Ade_corpus_v2_classification",
             "--model-name", "distilbert-base-uncased",
             "--train-ratio", "0.7",
             "--val-ratio", "0.15",]

In [14]:
hf_processor.run(
    code="scripts/preprocess.py",
    outputs=outputs,
    arguments=arguments
)


Job Name:  az-ade-2021-10-07-09-30-57-618
Inputs:  [{'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-905847418383/az-ade-2021-10-07-09-30-57-618/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-905847418383/az-ade-2021-10-07-09-30-57-618/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://az-ade-905847418383/processing_output/train_data', 'LocalPath': '/opt/ml/processing/training', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'validation_data', 'AppManaged': False, 'S3Output

In [15]:
preprocessing_job_description = hf_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])

s3://az-ade-905847418383/processing_output/train_data
s3://az-ade-905847418383/processing_output/validation_data
s3://az-ade-905847418383/processing_output/test_data
